In [1]:
import os
import gc
from copy import deepcopy

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = '../data'
data_filepath = os.path.join(data_dir, 'harddrive_nodup')
os.path.exists(data_filepath)

True

## Load data with PySpark session

In [3]:
spark = (SparkSession.builder
         .config("spark.driver.memory", "12g")
         .appName("SparkSQL").getOrCreate())

your 131072x1 screen size is bogus. expect trouble
24/08/24 12:08:28 WARN Utils: Your hostname, cedric-yu-work resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/24 12:08:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/24 12:08:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
date_col = 'date'
target_label = 'failure'
id_cols = ['serial_number', 'model']


In [5]:
df = (spark.read.option("header", "true")
      .option("inferSchema", "true")
      .csv(data_filepath))

In [13]:
df.show(10)

+----------+-------------+--------------------+-------------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+---

In [6]:
num_samples = df.count()

In [7]:
num_samples

3058798

In [8]:
gc.collect()

112

## Null values

Null values are 'NA'. Replace 'NA' by NULL

In [9]:
df = (df
      .select([F.when(F.col(c)=="NA", None)
               .otherwise(F.col(c))
               .alias(c)
               for c in df.columns]))

In [17]:
df.show(10)

+----------+-------------+--------------------+-------------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+---

In [10]:
na_count = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).cache()
na_frac = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).cache()

24/08/24 12:08:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/08/24 12:08:41 WARN CacheManager: Asked to cache already cached data.


In [11]:
na_count.show()

+----+-------------+-----+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+---------------

In [12]:
for field in na_frac.schema.fields:
    name = str(field.name)
    na_frac = na_frac.withColumn(name, F.col(name) / num_samples)


na_frac.show()

+----+-------------+-----+--------------+-------+------------------+-----------+------------------+------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+------------------+------------------+-----------+-------------------+------------+-------------------+-----------------+-------------------+------------+-------------------+------------------+-------------------+------------+-------------------+------------------+--------------------+-------------------+--------------------+------------------+--------------------+------------------+--------------------+-------------+--------------------+-------------------+--------------------+-----------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------

Number of columns to keep if we put cutoff on fraction of NULL elements

In [13]:
na_frac_cutoff = 0.4
cols_to_keep = [date_col, target_label] + id_cols
for key, value in na_frac.first().asDict().items():
    if value > na_frac_cutoff:
        continue
    elif key not in cols_to_keep:
        cols_to_keep.append(key)

In [14]:
len(cols_to_keep)

37

In [15]:
len(df.columns)

95

In [16]:
df1 = df.select(cols_to_keep)

## Constant columns

In [25]:
df1.agg(*(F.countDistinct(F.col(c)).alias(c) for c in df1.columns)).show()

+----+-------+-------------+-----+--------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+
|date|failure|serial_number|model|capacity_bytes|smart_1_normalized|smart_1_raw|smart_3_normalized|smart_3_raw|smart_4_normalized|smart_4_raw|smart_5_normalized|smart_5_raw|smart_7_normalized|smart_7_raw|smart_9_normalized|smart_9_raw|smart_10_normalized|smart_10_raw|smart_12_normalized|smart_12_raw|smart_188_raw|smart_192_normalized|smart_192_raw|smart_193_normalized|smart_193_raw|smart_194_normalized|smart_194_raw|smart_197_no

In [27]:
df1.groupby('smart_199_normalized').count().show()

+--------------------+-------+
|smart_199_normalized|  count|
+--------------------+-------+
|                 100|    440|
|                 200|3058358|
+--------------------+-------+



## Date range

In [22]:
print(df1.agg({date_col: "min"}).collect()[0])
print(df1.agg({date_col: "max"}).collect()[0])

Row(min(date)=datetime.date(2016, 1, 1))


Row(max(date)=datetime.date(2016, 4, 29))


In [28]:
df1 = df1.sort(id_cols + [date_col])

In [29]:
all_dates = df1.select(date_col).dropDuplicates().toPandas()

In [30]:
all_dates.sort_values(date_col)

,date
2,2016-01-01
37,2016-01-02
22,2016-01-03
40,2016-01-04
11,2016-01-05
12,2016-01-06
46,2016-01-07
32,2016-01-08
47,2016-01-09
44,2016-01-10


Data only in Jan and April 2016

Now look at date differences

In [17]:
datediff_by_id_cols = (df1.select(*(id_cols + [date_col]))
 .withColumn(
    date_col + "_lag_1", F.lag(date_col).over(Window.partitionBy(id_cols).orderBy(date_col))
)
.withColumn(
        "datediff",
        F.when(
            F.col(date_col + "_lag_1").isNotNull(),
            F.datediff(F.col(date_col), F.col(date_col + "_lag_1")),
        ).otherwise(F.lit(None)),
    )
    .drop(date_col + "_lag_1"))

In [18]:
datediff_by_id_cols.show()

+-------------+------------------+----------+--------+
|serial_number|             model|      date|datediff|
+-------------+------------------+----------+--------+
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-01|    NULL|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-02|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-03|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-04|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-05|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-06|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-07|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-08|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-09|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-10|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-11|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-12|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-13|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-14|       1|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-15|       1|
|    13H85

In [19]:
datediff_by_id_cols.dropna().groupby(id_cols).agg(F.countDistinct('datediff')).show()

+-------------+--------------------+------------------------+
|serial_number|               model|count(DISTINCT datediff)|
+-------------+--------------------+------------------------+
|    13H85BMGS|  TOSHIBA DT01ACA300|                       2|
|     2EG3NZ2P|HGST HUH728080ALE600|                       2|
|     2EG4B1VR|HGST HUH728080ALE600|                       2|
|     2EG4L4VP|HGST HUH728080ALE600|                       2|
| 4596K0X6FMYB| TOSHIBA MD04ABA400V|                       2|
| 45CAK0Q9FMYB| TOSHIBA MD04ABA400V|                       2|
| 45D7K132FMYB| TOSHIBA MD04ABA400V|                       2|
|     5VY50QZF|         ST3160318AS|                       2|
|     5XW004Q0|        ST31500541AS|                       1|
|     5XW00K1P|        ST31500541AS|                       1|
|     5XW00SQX|        ST31500541AS|                       1|
|     5XW00T9F|        ST31500541AS|                       1|
|     5XW01DHS|        ST31500541AS|                       1|
|     5X

In [20]:
df_cycle_id = (datediff_by_id_cols.fillna(1, subset='datediff').dropDuplicates(subset=id_cols + ['datediff'])
               .sort(id_cols + [date_col])
               .withColumn("cycle_id", F.row_number().over(Window().orderBy(F.lit('A')))))

In [21]:
df_cycle_id.show()

24/08/24 12:09:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+------------------+----------+--------+--------+
|serial_number|             model|      date|datediff|cycle_id|
+-------------+------------------+----------+--------+--------+
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-01|       1|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-04-01|      71|       2|
|    13H32WEAS|TOSHIBA DT01ACA300|2016-01-01|       1|       3|
|    13H32WEAS|TOSHIBA DT01ACA300|2016-04-01|      71|       4|
|    13H6A21GS|TOSHIBA DT01ACA300|2016-01-01|       1|       5|
|    13H6A21GS|TOSHIBA DT01ACA300|2016-04-01|      71|       6|
|    13H7X2HAS|TOSHIBA DT01ACA300|2016-01-01|       1|       7|
|    13H7X2HAS|TOSHIBA DT01ACA300|2016-04-01|      71|       8|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-01-01|       1|       9|
|    13H85BMGS|TOSHIBA DT01ACA300|2016-04-01|      71|      10|
|    13H87YWAS|TOSHIBA DT01ACA300|2016-01-01|       1|      11|
|    13H87YWAS|TOSHIBA DT01ACA300|2016-04-01|      71|      12|
|    13H883WAS|TOSHIBA DT01ACA300|2016-0

In [22]:
df_cycle_id2 = datediff_by_id_cols.fillna(1, subset='datediff').join(df_cycle_id,
 on=id_cols + [date_col, 'datediff'],
 how='left').sort(id_cols + [date_col]).cache()

24/08/24 12:09:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [23]:
df_cycle_id2.show()

24/08/24 12:09:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 1

+-------------+------------------+----------+--------+--------+
|serial_number|             model|      date|datediff|cycle_id|
+-------------+------------------+----------+--------+--------+
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-01|       1|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-02|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-03|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-04|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-05|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-06|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-07|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-08|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-09|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-10|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-11|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-12|       1|    NULL|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-0

In [24]:
df_cycle_id2 = df_cycle_id2.withColumn("cycle_id", F.last("cycle_id", ignorenulls=True).over(Window.orderBy(id_cols + [date_col]))).drop('datediff').cache()

24/08/24 12:09:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [25]:
df_cycle_id2.show()

24/08/24 12:09:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/24 12:09:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+------------------+----------+--------+
|serial_number|             model|      date|cycle_id|
+-------------+------------------+----------+--------+
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-01|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-02|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-03|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-04|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-05|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-06|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-07|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-08|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-09|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-10|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-11|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-12|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-13|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-14|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-15|       1|
|    13H2B

In [ ]:
df_cycle_id2.toPandas().to_csv('mycsv.csv')

## Target class distribution

In [26]:
df1.groupby([target_label]).count().show()

+-------+-------+
|failure|  count|
+-------+-------+
|      1|    205|
|      0|3058593|
+-------+-------+



In [15]:
205 / (205 + 3058593) * 100

0.0067019790126709895

Large class imbalance

## Model and serial numbers

In [12]:
df.agg(F.countDistinct(df['model'])).collect()

[Row(count(DISTINCT model)=69)]

In [13]:
df.select('model').distinct().show(10)

+--------------------+
|               model|
+--------------------+
|         ST9250315AS|
|         ST4000DM000|
|          ST320LT007|
|         ST3500320AS|
|      WDC WD5002ABYS|
|      WDC WD2500BEVT|
|Hitachi HDS5C3030...|
|        WDC WD60EFRX|
|       WDC WD800AAJB|
|Hitachi HDS722020...|
+--------------------+
only showing top 10 rows



In [16]:
df.agg(F.countDistinct(df['serial_number'])).toPandas()

,count(DISTINCT serial_number)
0,65993


In [17]:
df.select('serial_number').distinct().show(10)

+--------------+
| serial_number|
+--------------+
|JK11A8B9J7AAKF|
|PL1331LAGM029H|
|PL1331LAGR24XH|
|PL1331LAGTUP5H|
|      S300XDVP|
|      S300Z47R|
|      Z300CNRB|
|      Z300HLWD|
|      Z3010T5F|
|      Z3014A7H|
+--------------+
only showing top 10 rows



Number of serial number with failure

In [28]:
df1.agg(F.countDistinct('serial_number')).show()

+-----------------------------+
|count(DISTINCT serial_number)|
+-----------------------------+
|                        65993|
+-----------------------------+



In [27]:
df1.where(df1[target_label] == 1).agg(F.countDistinct('serial_number')).show()

+-----------------------------+
|count(DISTINCT serial_number)|
+-----------------------------+
|                          205|
+-----------------------------+



In [29]:
df_cycle_id2.agg(F.countDistinct('serial_number')).show()

+-----------------------------+
|count(DISTINCT serial_number)|
+-----------------------------+
|                        65993|
+-----------------------------+



Count of failed HDD by model

In [26]:
df1.where(df1[target_label] == 1).select(id_cols).dropDuplicates().groupby('model').count().sort('count', ascending=False).show()

+--------------------+-----+
|               model|count|
+--------------------+-----+
|         ST4000DM000|  131|
|          ST320LT007|   15|
|Hitachi HDS722020...|   13|
|       WDC WD800AAJS|    6|
|        WDC WD30EFRX|    6|
|Hitachi HDS5C3030...|    5|
|Hitachi HDS5C4040...|    3|
|HGST HMS5C4040ALE640|    2|
|      WDC WD1600AAJB|    2|
|         ST6000DX000|    2|
|         ST4000DX000|    2|
|      WDC WD1600AAJS|    2|
|        WDC WD20EFRX|    2|
|         ST3160318AS|    2|
|        WDC WD10EADS|    2|
|         ST9250315AS|    1|
|        WDC WD60EFRX|    1|
|       WDC WD800AAJB|    1|
|  TOSHIBA DT01ACA300|    1|
|      WDC WD3200BEKT|    1|
+--------------------+-----+
only showing top 20 rows



In [27]:
df1.where(df1[target_label] == 1).select(id_cols).dropDuplicates().groupby('serial_number').count().sort('count', ascending=False).show()

+---------------+-----+
|  serial_number|count|
+---------------+-----+
|       W300STTQ|    1|
|       Z300PMEL|    1|
|       W0Q26EE1|    1|
| JK1105B8J10GXX|    1|
|       Z300X5WP|    1|
| MJ1321YNG0JKMA|    1|
|       Z303RK24|    1|
|       Z304R6NY|    1|
|       Z30111VL|    1|
|       W0Q6WJGE|    1|
|       Z300ZQYJ|    1|
|       S300Z46R|    1|
|WD-WMC4N0727496|    1|
|       Z4D04BH9|    1|
|       Z302G5F6|    1|
| JK11A8B9J388ZF|    1|
|       Z30250YD|    1|
|       W300T1YZ|    1|
|       Z304JFAQ|    1|
|       Z304QCQL|    1|
+---------------+-----+
only showing top 20 rows



## Number of samples by serial number

In [51]:
df.groupby(id_cols).count().sort('count', ascending=False).show(10)

+---------------+--------------------+-----+
|  serial_number|               model|count|
+---------------+--------------------+-----+
| MJ1311YNG7BH3A|Hitachi HDS5C3030...|   52|
|       S300Z6K9|         ST4000DM000|   52|
|       S300ZVMZ|         ST4000DM000|   52|
|       S3010TR7|         ST4000DM000|   52|
| PL2331LAGM7YGJ|HGST HMS5C4040BLE640|   52|
|       S30088X7|         ST4000DM000|   52|
|       Z305BT2T|         ST4000DM000|   52|
| PL1331LAGT94SH|HGST HMS5C4040ALE640|   52|
|WD-WX41D94RNV69|        WDC WD60EFRX|   52|
| JK11A8B9J9KYXF|Hitachi HDS722020...|   52|
+---------------+--------------------+-----+
only showing top 10 rows



## Statistics

In [41]:
num_cols = [c for c in df1.columns if c not in [date_col, target_label] + id_cols]

In [42]:
df1.first()

Row(date=datetime.date(2016, 1, 1), failure=0, serial_number='13H2B97AS', model='TOSHIBA DT01ACA300', capacity_bytes=1.4824899095664e-311, smart_1_normalized=99, smart_1_raw=131072, smart_3_normalized=134, smart_3_raw=429, smart_4_normalized=100, smart_4_raw=18, smart_5_normalized=100, smart_5_raw=0, smart_7_normalized=100, smart_7_raw=0.0, smart_9_normalized=98, smart_9_raw=16274, smart_10_normalized=100, smart_10_raw=0, smart_12_normalized=100, smart_12_raw=18, smart_188_raw=0.0, smart_192_normalized='100', smart_192_raw='26', smart_193_normalized='100', smart_193_raw='26', smart_194_normalized='187', smart_194_raw='32', smart_197_normalized=100, smart_197_raw=0, smart_198_normalized=100, smart_198_raw=0, smart_199_normalized=200, smart_199_raw=10, smart_240_raw=0.0, smart_241_raw=0.0, smart_242_raw=0.0)

In [43]:
df1.describe().show()

+-------+-------------------+-------------+--------------------+--------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+-----------------+-------------------+------------------+--------------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+-------------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+
|summary|            failure|serial_number|               model|      capacity_bytes|smart_1_normalized|        smart_1_raw|smart_3_normalized|       smart_3_raw| smart_4_normalized|       smart_4_raw|smart_5_normalized|       smart_5_raw|smart_7_normalized|         smart_7_raw|sma

In [44]:
mean_by_class = df1.drop(*[date_col] + id_cols).groupby(target_label).mean()

In [45]:
mean_by_class.show()

+-------+------------+--------------------+-----------------------+-------------------+-----------------------+------------------+-----------------------+----------------+-----------------------+-----------------+-----------------------+-----------------+-----------------------+------------------+------------------------+--------------------+------------------------+------------------+------------------+-------------------------+-------------------+-------------------------+------------------+-------------------------+------------------+--------------------+------------------+------------------+
|failure|avg(failure)| avg(capacity_bytes)|avg(smart_1_normalized)|   avg(smart_1_raw)|avg(smart_3_normalized)|  avg(smart_3_raw)|avg(smart_4_normalized)|avg(smart_4_raw)|avg(smart_5_normalized)| avg(smart_5_raw)|avg(smart_7_normalized)| avg(smart_7_raw)|avg(smart_9_normalized)|  avg(smart_9_raw)|avg(smart_10_normalized)|   avg(smart_10_raw)|avg(smart_12_normalized)| avg(smart_12_raw)|avg(smart

## Only keep cycles that eventually run to failure

In [33]:
df_cycle_id_failure = (df_cycle_id2.join(
    df1.select(*(id_cols + [date_col, target_label])),
    on=id_cols + [date_col],
    how='left')
    .drop(*(id_cols + [date_col])))
df_cycle_id_failure = (df_cycle_id_failure
                       .where(df_cycle_id_failure['failure'] == 1)
                       .drop('failure')
                       .dropDuplicates().sort('cycle_id'))
# df_cycle_id_failure = df_cycle_id2.where(df_cycle_id2['cycle_id'].isin(df_cycle_id_failure['cycle_id']))

In [46]:
df_cycle_id2.join(df_cycle_id_failure,
                  on='cycle_id', how='inner').count()

2757

In [41]:
df_cycle_id2.where(df_cycle_id2['cycle_id'].isin(df_cycle_id_failure['cycle_id'])).agg(F.countDistinct('cycle_id')).show()

+------------------------+
|count(DISTINCT cycle_id)|
+------------------------+
|                  123072|
+------------------------+



In [36]:
df_cycle_id2.agg(F.countDistinct('cycle_id')).show()

+------------------------+
|count(DISTINCT cycle_id)|
+------------------------+
|                  123072|
+------------------------+



In [35]:
df_cycle_id_failure.count()

205

In [34]:
df_cycle_id_failure.show()

+--------+
|cycle_id|
+--------+
|      14|
|     559|
|     825|
|     827|
|    1340|
|    1927|
|    2176|
|    3043|
|    3588|
|    3594|
|    3605|
|    4323|
|    5635|
|    5905|
|    6514|
|    9017|
|    9623|
|   14154|
|   14446|
|   14841|
+--------+
only showing top 20 rows



In [32]:
df_cycle_id_failure.agg(F.countDistinct('serial_number')).show()

+-----------------------------+
|count(DISTINCT serial_number)|
+-----------------------------+
|                        65993|
+-----------------------------+



In [67]:
df1 = df1.join(df_cycle_id_failure,
         on=id_cols + [date_col],
         how='left').sort(id_cols + [date_col])

In [68]:
df1.show(20)

+-------------+------------------+----------+-------+--------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+--------+--------+
|serial_number|             model|      date|failure|      capacity_bytes|smart_1_normalized|smart_1_raw|smart_3_normalized|smart_3_raw|smart_4_normalized|smart_4_raw|smart_5_normalized|smart_5_raw|smart_7_normalized|smart_7_raw|smart_9_normalized|smart_9_raw|smart_10_normalized|smart_10_raw|smart_12_normalized|smart_12_raw|smart_188_raw|smart_192_normalized|smart_192_raw|smart_193_norm

In [47]:
spark.stop()